In [1]:
import tensorflow as tf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline 
from functions import MLP_generator,createDataMain,createTestDataMain
import cPickle

In [2]:
batch_size = 100
seq_len = 16 # same as test set for convenience
learning_rate = 1e-4
epoch = 10

In [3]:
# time dependent list of discrete variables of shape (B,T)
discreteList = ['dayOfWeek','payDay','month','earthquake','type','locale','locale_name','transferred','onpromotion']
# time independent list of discrete variables of shape of shape (B,)
disc_X = ['store_nbr','item_nbr',
 'family', 'class',
 'perishable', 'city',
 'state', 'type', 'cluster']

In [4]:
cardinalitys_X = [55, 4001, 34, 337, 2, 23, 17, 6, 18]
cardinalitys_T = [7, 2, 13, 2, 7, 4, 25, 2, 3]

In [5]:
dimentions_X = [2, 20, 1, 2, 1, 1, 1, 1, 1]
dimentions_T = [1, 1, 1, 1, 1, 1, 1, 1, 1]

In [6]:
dX = sum(dimentions_X)
dT = sum(dimentions_T)
d = 50

In [7]:
lambdas = np.array([2, 2, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]) * 1e-4 # regularization coefficient
l1,l2,l3 = 1e-4,1e-4,1e-4

** Build Computational Graph **

In [8]:
def createGraph(batch_size,seq_len,cardinalitys_X,cardinalitys_T,\
                dimentions_X,dimentions_T,dX,dT,d,lambdas,l1,l2,l3):
    tf.reset_default_graph()
    embedding_X = [tf.get_variable("embedding_X"+str(i), [car, dim],\
                                   initializer=tf.truncated_normal_initializer()) \
                for i,(car,dim) in enumerate(zip(cardinalitys_X,dimentions_X))]
    embedding_Xt = [tf.get_variable("embedding_Xt"+str(i), [car, dim],\
                                    initializer=tf.truncated_normal_initializer()) \
                    for i,(car,dim) in enumerate(zip(cardinalitys_T,dimentions_T))]
    embeddings = embedding_X + embedding_Xt
    learning_rate = tf.placeholder(tf.float32,shape=[])
    X = [tf.placeholder(tf.int32, [batch_size,], name='X_'+str(i)) for i,_ in enumerate(dimentions_X)]
    Xt = [tf.placeholder(tf.int32, [batch_size,seq_len], name='Xt_'+str(i)) for i,_ in enumerate(dimentions_T)]
    X_continuous = tf.placeholder(tf.float32, [batch_size,seq_len,1], name='X_continuous')
    Weight = tf.placeholder(tf.float32, [batch_size,seq_len], name='Weight')
    y = tf.placeholder(tf.float32, [batch_size,seq_len], name='y')
    inputs = [y,Weight,X_continuous] + Xt + X + [learning_rate]
    Xt1 = tf.concat([tf.nn.embedding_lookup(emb,x) for emb,x in zip(embedding_Xt,Xt)] + [X_continuous],2)
    X1 = tf.concat([tf.nn.embedding_lookup(emb,x) for emb,x in zip(embedding_X,X)],1)
    weights1_X = tf.Variable(tf.truncated_normal([dX,d],
                        stddev=1.0 / np.sqrt(dX)),name='weights1_X')
    weights1_Xt = tf.Variable(tf.truncated_normal([dT+1,d],
                            stddev=1.0 / np.sqrt(dT)),name='weights1_Xt')
    biases1 = tf.Variable(tf.zeros([d]),
                         name='biases1')
    X2 = tf.nn.relu(biases1 + \
                   tf.reshape(tf.matmul(X1,weights1_X),(-1 if batch_size is None else batch_size,1,d))+\
                   tf.einsum('btp,pq->btq', Xt1, weights1_Xt))
    weights2 = tf.Variable(tf.truncated_normal([d,d],
                        stddev=1.0 / np.sqrt(d)),name='weights2')
    biases2 = tf.Variable(tf.zeros([d]),
                         name='biases2')
    X3 = tf.nn.relu(tf.einsum('btp,pq->btq', X2, weights2) + biases2)
    weights3 = tf.Variable(tf.truncated_normal([d],
                        stddev=1.0 / np.sqrt(d)),name='weights3')
    biases3 = tf.Variable(tf.zeros([1]),
                         name='biases3')
    yhat = tf.nn.relu(tf.einsum('btp,p->bt', X3, weights3) + biases3) # as target is always positive
    regularizer = sum([l*tf.reduce_sum(v**2) for l,v in zip(lambdas,embeddings)])\
                + l1 * tf.reduce_sum(weights1_X**2) + l1 * tf.reduce_sum(weights1_Xt**2)\
                + l2 * tf.reduce_sum(weights2**2)\
                + l3 * tf.reduce_sum(weights3**2)
    cost = tf.reduce_mean(Weight*(tf.log((yhat+1)/(y+1)))**2)
    augment_cost = cost + regularizer
    train_op = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(augment_cost)
    saver = tf.train.Saver()  
    return inputs,train_op,cost,saver,yhat    

In [ ]:
inputs,train_op,cost,saver,yhat = createGraph(batch_size,seq_len,cardinalitys_X,cardinalitys_T,\
                                        dimentions_X,dimentions_T,dX,dT,d,lambdas,l1,l2,l3)

** Training **

In [18]:
sess = tf.InteractiveSession()
sess.run(tf.global_variables_initializer())

In [19]:
monitor = 10000

In [9]:
with open(r"dateVar.pickle", "rb") as input_file:
    dateVar_list = cPickle.load(input_file)

In [10]:
train,val = createDataMain(True)

/home/will/anaconda2/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2821: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):


In [212]:
for i in range(epoch):
    cost_train = .0
    weight_train = .0
    cost_val = .0
    weight_val = .0
    for j,X_nps in enumerate(MLP_generator(train,batch_size,seq_len,\
                                           dateVar_list,'date',discreteList,True,0.95)):
        _,cost_np = sess.run([train_op,cost],dict(zip(inputs,X_nps+[learning_rate])))
        cost_train += cost_np
        weight_train = weight_train + np.sum(X_nps[1])
        
        if j%monitor == 0:
            for X_nps in MLP_generator(val,batch_size,seq_len,\
                                       dateVar_list,'date',discreteList,False,0.95):
                cost_np = sess.run([cost],dict(zip(inputs,X_nps)))
                cost_val += cost_np[0]       
                weight_val = weight_val + np.sum(X_nps[1])
        
            print "Train loss:{}, Val loss:{}".format(np.sqrt(seq_len*batch_size*cost_train/weight_train),\
                                                      np.sqrt(seq_len*batch_size*cost_val/weight_val))
            cost_train = .0
            weight_train = .0
            cost_val = .0
            weight_val = .0

Train loss:1.1847664692, Val loss:1.60325047916
Train loss:0.986747555891, Val loss:1.07117274607
Train loss:0.879173041289, Val loss:1.0591551266
Train loss:1.00505547683, Val loss:1.03617483385
Train loss:0.945495119286, Val loss:1.02166263329
Train loss:0.967008211188, Val loss:1.01747927126
Train loss:0.880663006896, Val loss:0.997249728902
Train loss:0.960428059044, Val loss:0.980836555935
Train loss:0.807946631411, Val loss:0.951968467435
Train loss:0.813474913948, Val loss:0.940263721327


In [215]:
# Save model parameters
saver.save(sess,'ANN_fillin')

'ANN_fillin'

** Prediction **

In [9]:
inputs,train_op,cost,saver,yhat = createGraph(None,seq_len,cardinalitys_X,cardinalitys_T,\
                                              dimentions_X,dimentions_T,dX,dT,d,lambdas,l1,l2,l3)

In [11]:
sess = tf.InteractiveSession()
sess.run(tf.global_variables_initializer())

In [12]:
saver.restore(sess,'ANN_fillin')

INFO:tensorflow:Restoring parameters from ANN_fillin


In [18]:
test_SI,test_newItem = createTestDataMain()

In [42]:
range(0,10,3)

[0, 3, 6, 9]

In [60]:
test_New#.shape[0]

,store_nbr,item_nbr,id,family,class,perishable,city,state,type,cluster,onpromotion,amin,amax
0,1,313094,"(125497336, 125707990, 125918644, 126129298, 1...",1,32,0,1,1,1,13,"(0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0)",2017-08-16,2017-08-31
1,1,1464607,"(125499597, 125710251, 125920905, 126131559, 1...",9,250,0,1,1,1,13,"(0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0)",2017-08-16,2017-08-31
2,1,1467088,"(125499616, 125710270, 125920924, 126131578, 1...",1,86,0,1,1,1,13,"(0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0)",2017-08-16,2017-08-31
3,1,1467093,"(125499617, 125710271, 125920925, 126131579, 1...",1,1,0,1,1,1,13,"(0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0)",2017-08-16,2017-08-31
4,1,2011437,"(125500528, 125711182, 125921836, 126132490, 1...",1,320,0,1,1,1,13,"(0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0)",2017-08-16,2017-08-31
5,1,2011448,"(125500529, 125711183, 125921837, 126132491, 1...",1,320,0,1,1,1,13,"(0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0)",2017-08-16,2017-08-31
6,1,2011451,"(125500532, 125711186, 125921840, 126132494, 1...",1,320,0,1,1,1,13,"(0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0)",2017-08-16,2017-08-31
7,1,2011457,"(125500533, 125711187, 125921841, 126132495, 1...",1,320,0,1,1,1,13,"(0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0)",2017-08-16,2017-08-31
8,1,2011459,"(125500534, 125711188, 125921842, 126132496, 1...",1,320,0,1,1,1,13,"(0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0)",2017-08-16,2017-08-31
9,1,2011468,"(125500536, 125711190, 125921844, 126132498, 1...",1,320,0,1,1,1,13,"(0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0)",2017-08-16,2017-08-31


In [65]:
data = test_New.head()

In [67]:
sparse = pd.concat([data.apply(lambda x: pd.Series(x.id),axis=1)\
                                .stack().reset_index().drop(['level_1'],1),\
                    data.apply(lambda x: pd.Series(x.onpromotion),axis=1)\
                                .stack().reset_index().drop(['level_0','level_1'],1)],1)

In [69]:
data

,store_nbr,item_nbr,id,family,class,perishable,city,state,type,cluster,onpromotion,amin,amax
0,1,313094,"(125497336, 125707990, 125918644, 126129298, 1...",1,32,0,1,1,1,13,"(0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0)",2017-08-16,2017-08-31
1,1,1464607,"(125499597, 125710251, 125920905, 126131559, 1...",9,250,0,1,1,1,13,"(0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0)",2017-08-16,2017-08-31
2,1,1467088,"(125499616, 125710270, 125920924, 126131578, 1...",1,86,0,1,1,1,13,"(0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0)",2017-08-16,2017-08-31
3,1,1467093,"(125499617, 125710271, 125920925, 126131579, 1...",1,1,0,1,1,1,13,"(0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0)",2017-08-16,2017-08-31
4,1,2011437,"(125500528, 125711182, 125921836, 126132490, 1...",1,320,0,1,1,1,13,"(0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0)",2017-08-16,2017-08-31


In [68]:
sparse

,level_0,0,0
0,0,125497336,0
1,0,125707990,0
2,0,125918644,0
3,0,126129298,0
4,0,126339952,0
5,0,126550606,0
6,0,126761260,0
7,0,126971914,0
8,0,127182568,0
9,0,127393222,0
